In [ ]:
# python2 and python3 compatibility between loaded modules
from __future__ import print_function

In [ ]:
# All imports go here
%matplotlib inline

# Reading files
import os

# Vector manipulations
import numpy as np

# Torch - DL framework
import torch
from torch import nn
from torch.nn import functional as F
import torch.utils.data as td
import torchvision as tv

# Plotting images
from matplotlib import pyplot as plt

# COCO loading captions
from pycocotools.coco import COCO
import skimage.io as io
import pylab
pylab.rcParams['figure.figsize'] = (8.0, 10.0)

"""
# evaluation metrics on MSCOCO dataset
from pycocoevalcap.eval import COCOEvalCap
from pycocoevalcap.tokenizer.ptbtokenizer import PTBTokenizer
from pycocoevalcap.bleu.bleu import Bleu
from pycocoevalcap.meteor.meteor import Meteor
from pycocoevalcap.rouge.rouge import Rouge
from pycocoevalcap.cider.cider import Cider
from pycocoevalcap.spice.spice import Spice
"""

# json for dumping stuff onto files as output
import json
from json import encoder
encoder.FLOAT_REPR = lambda o: format(o, '.3f')

In [ ]:
# select the relevant device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

In [ ]:
# data loading
dataset_root_dir = '/datasets/COCO-2015/'
annotations_root_dir = '../datasets/COCO/annotations/'
train_dir = "train2014"
val_dir = "val2014"
test_dir = "test2015"

In [ ]:
# training data annotations
train_ann = "{}captions_{}.json".format(annotations_root_dir, train_dir)
coco_train_caps = COCO(train_ann)

In [ ]:
help(coco_train_caps)

In [ ]:
# get all the image IDs
# Not required
train_image_ids = coco_train_caps.getImgIds()
# loadImgs() returns all the images
train_imgs = coco_train_caps.loadImgs(train_image_ids)

In [ ]:
print(type(train_imgs), len(train_imgs))

In [ ]:
# read 1 random image from file
rand_id = np.random.randint(0, len(train_imgs))
rand_img = io.imread('{}/{}/{}'.format(dataset_root_dir, train_dir, imgs[rand_id]['file_name']))

In [ ]:
plt.axis('off')
plt.imshow(rand_img)
plt.show()

In [ ]:
# load caption for this particular image
ann_id = coco_train_caps.getAnnIds(imgIds=train_imgs[rand_id]['id'])
anns = coco_train_caps.loadAnns(ann_id)
coco_train_caps.showAnns(anns)
f = plt.figure()
plt.imshow(rand_img)
plt.axis("off")
plt.show()